<a href="https://colab.research.google.com/github/margaret-tk/APS360-AI/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import svm, metrics

clf = svm.LinearSVC(max_iter=2000)
#clf = svm.SVC(gamma = 0.001)
clf.fit(train_images,train_labels)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
# location on Google Drive
master_path = '/content/drive/My Drive/dataset/imgs/'

classes = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']

data_transform = transforms.Compose([transforms.RandomResizedCrop(64), 
                                      transforms.ToTensor()])

dataset = datasets.ImageFolder(master_path, transform=data_transform)

In [0]:
train_index, test_index = int(0.7*len(dataset)), int(0.3*len(dataset))+1
train, test = torch.utils.data.random_split(dataset, [train_index,test_index])

In [0]:
print(len(test))
print(len(train))

8102
18902


In [0]:
# define dataloader parameters
batch_size  = 500
num_workers = 0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=5000, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=500, 
                                           num_workers=num_workers, shuffle=True)

In [0]:
# Visualize some sample data

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() 
# convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

KeyboardInterrupt: ignored

In [0]:
images, labels = dataiter.next()
alexNet = torchvision.models.alexnet(pretrained=True)
features = alexNet.features(images)
features.shape

torch.Size([20, 256, 1, 1])

In [0]:
def get_accuracy(model, train=False):
    if train:
        data_loader = train_loader
    else:
        data_loader = val_loader

    correct = 0
    total = 0
    for imgs, labels in data_loader:
        
        imgs = alexNet.features(imgs) #SLOW
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [0]:
def train(model, train_loader, batch_size=20, num_epochs=30):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
          
            imgs = features = alexNet.features(imgs)       
              
            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            train_acc.append(get_accuracy(model, train=True)) # compute training accuracy 
            val_acc.append(get_accuracy(model, train=False))  # compute validation accuracy
            n += 1

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [0]:
dataiter = iter(train_loader)
train_images, train_labels = dataiter.next()
train_images = train_images.numpy()
#val
dataiter = iter(test_loader)
test_images, test_labels = dataiter.next()
test_images = test_images.numpy()



In [0]:
from sklearn import svm, metrics, preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
rescaled_features_train = scaler.fit_transform(train_images)
rescaled_features_test = scaler.fit_transform(test_images)

In [0]:
train_images = np.reshape(rescaled_features_train, (train_images.shape[0], -1))
test_images = np.reshape(rescaled_features_test, (test_images.shape[0], -1))

In [0]:
#from sklearn import svm, metrics

#clf = svm.LinearSVC(max_iter=2000)
clf = svm.SVC()
clf.fit(train_images,train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(test_images)
print(y_pred)

[7 7 5 5 4 6 4 7 1 0 4 9 2 8 5 9 4 9 3 2 7 7 9 3 7 4 5 6 0 3 4 2 2 4 2 2 2
 7 0 5 2 2 5 7 7 9 7 9 0 1 7 6 7 9 9 4 2 0 1 5 8 7 3 4 1 7 5 0 3 0 6 5 4 9
 4 1 1 2 9 6 8 3 1 6 2 7 8 6 1 6 1 2 2 7 4 0 7 0 4 3 1 0 7 4 2 1 8 2 9 7 7
 1 2 4 1 0 6 0 2 2 7 2 5 0 5 0 7 1 1 3 2 7 4 7 2 7 0 1 1 1 6 9 7 8 3 7 1 7
 8 7 0 7 2 7 9 0 4 7 5 1 1 7 1 1 9 5 9 5 9 5 2 1 2 4 4 5 5 7 9 9 6 5 5 3 2
 9 1 1 0 8 3 0 8 5 5 2 2 4 7 6 2 0 5 8 8 8 7 6 4 0 2 2 0 7 5 9 7 2 7 7 2 5
 4 4 0 1 8 8 2 8 9 1 5 5 7 1 1 2 7 4 0 6 9 1 7 7 6 8 4 7 1 1 7 4 1 7 5 3 3
 0 5 2 0 2 4 9 4 8 0 0 2 6 9 7 0 4 0 6 2 0 5 5 7 7 4 1 2 0 7 2 1 4 8 1 8 9
 2 2 2 0 4 1 0 4 2 3 3 2 2 6 1 9 1 5 4 3 4 2 9 6 9 2 4 1 8 8 1 1 0 0 6 0 2
 8 2 2 7 8 4 7 7 8 0 7 0 7 1 4 2 2 1 8 0 1 0 9 7 2 0 9 5 8 9 6 6 0 7 1 6 4
 0 2 0 7 8 5 7 2 6 1 4 9 6 2 6 4 7 7 1 0 7 2 1 7 7 7 4 6 1 9 8 7 7 6 5 7 2
 1 2 9 0 0 7 6 1 8 1 6 1 5 2 7 5 1 0 1 2 7 4 3 2 6 1 9 9 1 8 1 0 0 6 1 1 0
 4 1 5 2 6 1 2 7 1 4 2 6 7 4 1 7 1 3 5 8 6 1 2 2 8 5 2 2 2 9 6 0 6 7 9 5 0
 2 7 8 7 2 6 2 1 7 7 8 3 

In [0]:
print("Classification report for - \n{}:\n{}\n".format(clf, metrics.classification_report(test_labels, y_pred)))

Classification report for - 
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       0.49      0.45      0.47        60
           1       0.70      0.94      0.80        52
           2       0.40      0.48      0.43        63
           3       0.37      0.14      0.20        50
           4       0.73      0.86      0.79        42
           5       0.63      0.70      0.67        37
           6       0.58      0.46      0.51        48
           7       0.49      0.64      0.55        61
           8       0.50      0.44      0.47        39
           9       0.95      0.77      0.85        48

    accuracy                           0.58       500
   macro avg       0.58      0.59      0.57       500
weighted avg    

In [0]:
print(test_labels)

tensor([0, 3, 9, 9, 7, 8, 4, 9, 0, 6, 7, 1, 8, 1, 4, 2, 3, 7, 9, 5, 8, 4, 5, 9,
        8, 4, 5, 7, 1, 6, 5, 6, 2, 5, 0, 0, 6, 4, 3, 8, 3, 1, 7, 7, 2, 8, 7, 4,
        3, 9, 7, 0, 2, 9, 3, 5, 9, 3, 8, 4, 0, 2, 8, 1, 2, 0, 8, 8, 2, 5, 3, 6,
        1, 4, 6, 6, 1, 1, 7, 2, 2, 3, 1, 2, 6, 3, 7, 1, 6, 8, 1, 5, 2, 0, 9, 5,
        0, 1, 8, 9])
